### Todos:
### 1. preprocess the data to make it suitable for glove embeding:
### a. remove ' from words( don't -> dont)
### b. convert all letters to lowercase ( Hoping -> hoping)
### 2. find the frequency of each emoji to see if the dataset is balanced and balance the dataset(important)

In [1]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
import interesting_labels


In [2]:
import pandas

In [168]:
def get_data():
    header = ['status_id','tweet','label']
    data_set = pandas.read_csv('processed.txt',delimiter='\t',names = header)
    return data_set

def get_data_set():
    return global_data_set

def get_glove_embedding(glove): 
    return nn.Embedding.from_pretrained(glove.vectors)

def get_label_mapping():
    original_list = interesting_labels.wanted_list
    output_list = [i for i in range(len(original_list))]
    return dict(zip(original_list,output_list))
  
def cleanup_tweet(tweet):
    if(type(tweet) == str):
        tweet = tweet.lower()
        tweet = tweet.replace('\'','')
    return tweet
    
# How many of each category there are
# currently: {'Happy': 2097, 'Sad': 938, 'Angry': 235, 'Surprised': 234, 'Disgusted': 64, 'Afraid': 24}
def get_total_each_category(data):
    total = {"Happy":0, "Sad":0, "Angry":0, "Surprised":0, "Disgusted":0, "Afraid":0}
    for i in range(len(data)):
        if (data['label'][i] in interesting_labels.Happy):
            total['Happy'] += 1
        elif (data['label'][i] in interesting_labels.Sad):
            total['Sad'] += 1
        elif (data['label'][i] in interesting_labels.Angry):
            total['Angry'] += 1
        elif (data['label'][i] in interesting_labels.Surprised):
            total['Surprised'] += 1
        elif (data['label'][i] in interesting_labels.Disgusted):
            total['Disgusted'] += 1
        elif (data['label'][i] in interesting_labels.Afraid):
            total['Afraid'] += 1
    print (total)

# Right now not generalized to all dataset shapes
# At the moment will only change 1 word in sad, and the rest as many as possible
# to get as many up to same shape
def change_to_synonyms(data):
    for i in range(len(data)):
        if (data['label'][i] in interesting_labels.Sad):
            string = data['tweet'][i]
            # Need to change 1 synonym, randomly choose in each string
            new_data = pandas.DataFrame({'status_id': pandas.Series(data['status_id'], dtype=np.int64),
                                         'tweet': pandas.Series(data['tweet'], dtype=str),
                                         'label': pandas.Series(data['label'], dtype=np.int64)})
            data = data.append(new_data, ignore_index=True)
        '''elif (data['label'][i] in interesting_labels.Angry):
            for j, string in enumerate(data['tweet'][i]):
                new_data = {data['status_id'][i]: (string, data['label'][i])}
                data.append(new_data, ignore_index=True)
        
        elif (data['label'][i] in interesting_labels.Surprised):
            for j, string in enumerate(data['tweet'][i]):
                # string needs to equal the synoymns
                new_data = {data['status_id'][i]: (string, data['label'][i])}
                data.append(new_data, ignore_index=True)
        
        elif (data['label'][i] in interesting_labels.Disgusted):
            for j, string in enumerate(data['tweet'][i]):
                new_data = {data['status_id'][i]: (string, data['label'][i])}
                data.append(new_data, ignore_index=True)
        
        elif (data['label'][i] in interesting_labels.Afraid):
            for j, string in enumerate(data['tweet'][i]):
                new_data = {data['status_id'][i]: (string, data['label'][i])}
                data.append(new_data, ignore_index=True)'''
    print(data.info())
    return data

In [169]:
mapping = get_label_mapping()

In [170]:
global_data_set = get_data()

In [171]:
glove = torchtext.vocab.GloVe(name="twitter.27B",dim=50)

In [172]:
glove_emb = get_glove_embedding(glove)

In [173]:
data_set = get_data_set()

In [174]:
len(data_set)

3592

In [175]:
data_set['label'][0]

1397

In [176]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3592 entries, 0 to 3591
Data columns (total 3 columns):
status_id    3592 non-null int64
tweet        3592 non-null object
label        3592 non-null int64
dtypes: int64(2), object(1)
memory usage: 84.3+ KB


In [177]:
get_total_each_category(data_set)
count_sad(data_set)

{'Happy': 2097, 'Sad': 938, 'Angry': 235, 'Surprised': 234, 'Disgusted': 64, 'Afraid': 24}
938


In [178]:
data_set = change_to_synonyms(data_set)

KeyboardInterrupt: 

In [ ]:
get_total_each_category(data_set)
count_sad(data_set)

In [108]:
print(type(data_set['status_id'][0]))
print(type(data_set['tweet'][0]))
print(type(data_set['label'][0]))
print(type(data_set['status_id'][4521]))
print(type(data_set['tweet'][4521]))
print(type(data_set['label'][4521]))

<class 'int'>
<class 'str'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>


In [15]:
data_set

,status_id,tweet,label
0,744211689319501825,"Like hello, I'm watching it too. Mind lowering...",1397
1,742516659219509252,can u Dm ur pics with the twins for my ig fan ...,1380
2,746762559387111425,happy birthday jaz. Enjoy your day,623
3,743395873757892609,got joe and pewds book soon tabinof and da sid...,1380
4,747399941434146817,"Every time I up this late, I'm usually always ...",1381
5,744374136793759744,I can't wait to meet my stepbrother next year.,1384
6,747927031728377857,When you know you can't sing but you're singin...,1389
7,742575673332531200,I love that's daddy forever,1420
8,747312310121697280,We are lonely,675
9,747090755890577409,Sleepy in church,1399


In [ ]:
type(data_set['tweet'][0])

In [ ]:
data_set

In [ ]:
data_set['label']=data_set['label'].map(mapping)

In [ ]:
data_set

In [ ]:
data_set['tweet'] = data_set['tweet'].apply(cleanup_tweet)

In [ ]:
data_set

In [ ]:
def split_tweet(tweet):
    tweet = tweet.replace(".", " . ") \
                 .replace(",", " , ") \
                 .replace(";", " ; ") \
                 .replace("?", " ? ")
    return tweet.split()

In [ ]:
def get_tweet_words(glove_vector):
    train, valid, test = [],[],[]
    data_set = get_data_set()
    for i in range(len(data_set)):
        tweet = data_set['tweet'][i]
        if(type(tweet) != str):
            continue
        idxs = [glove_vector.stoi[w]        # lookup the index of word
                for w in split_tweet(tweet)
                if w in glove_vector.stoi] # keep words that has an embedding
        if not idxs: # ignore tweets without any word with an embedding
            continue
        idxs = torch.tensor(idxs) # convert list to pytorch tensor
        label = torch.tensor(data_set['label'][i]).long()
        if i % 5 < 3:
            train.append((idxs, label))
        elif i % 5 == 4:
            valid.append((idxs, label))
        else:
            test.append((idxs, label))
    return train, valid, test

In [ ]:
train, valid, test = get_tweet_words(glove)

In [ ]:
train[0]

In [ ]:
valid[0]

In [ ]:
len(train)

In [ ]:
tweet_emb = glove_emb(train[0][0])
tweet_emb.shape

### Recurrent Neural Network Module

In [ ]:
rnn_layer = nn.RNN(input_size=50,    # dimension of the input repr
                   hidden_size=50,   # dimension of the hidden units
                   batch_first=True) # input format is [batch_size, seq_len, repr_dim]

In [ ]:
tweet_input = tweet_emb.unsqueeze(0) # add the batch_size dimension
h0 = torch.zeros(1, 1, 50)     # initial hidden layer
out, last_hidden = rnn_layer(tweet_input, h0)

In [ ]:
print(out.shape)
print(last_hidden.shape)

In [ ]:
out[:,-1,:]

### Building the model

In [ ]:
class TweetRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TweetRNN, self).__init__()
        self.emb = nn.Embedding.from_pretrained(glove.vectors)
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Look up the embedding
        x = self.emb(x)
        # Set an initial hidden state
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out

model = TweetRNN(input_size=50, hidden_size=50, num_classes=len(interesting_labels.wanted_list))

In [ ]:
data_set.nunique()

In [ ]:
train[0:16097][1]

In [ ]:
len(set([element[1].item() for element in train]))

In [ ]:
import random

class TweetBatcher:
    def __init__(self, tweets, batch_size=32, drop_last=False):
        # store tweets by length
        self.tweets_by_length = {}
        for words, label in tweets:
            # compute the length of the tweet
            wlen = words.shape[0]
            # put the tweet in the correct key inside self.tweet_by_length
            if wlen not in self.tweets_by_length:
                self.tweets_by_length[wlen] = []
            self.tweets_by_length[wlen].append((words, label),)
         
        #  create a DataLoader for each set of tweets of the same length
        self.loaders = {wlen : torch.utils.data.DataLoader(
                                    tweets,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    drop_last=drop_last) # omit last batch if smaller than batch_size
            for wlen, tweets in self.tweets_by_length.items()}
        
    def __iter__(self): # called by Python to create an iterator
        # make an iterator for every tweet length
        iters = [iter(loader) for loader in self.loaders.values()]
        while iters:
            # pick an iterator (a length)
            im = random.choice(iters)
            try:
                yield next(im)
            except StopIteration:
                # no more elements in the iterator, remove it
                iters.remove(im)

In [ ]:
for i, (tweets, labels) in enumerate(TweetBatcher(train, drop_last=True)):
    print(tweets.shape, labels.shape)
print(i)

In [ ]:
def get_accuracy(model, data_loader):
    correct, total = 0, 0
    for tweets, labels in data_loader:
        output = model(tweets)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
    return float(correct) / float(total)

test_loader = TweetBatcher(test, batch_size=32, drop_last=False)
get_accuracy(model, test_loader)


In [ ]:
def train_rnn_network(model, train, valid, num_epochs=5, learning_rate=1e-5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses, train_acc, valid_acc = [], [], []
    epochs = []
    for epoch in range(num_epochs):
        for tweets, labels in train:
            optimizer.zero_grad()
            pred = model(tweets)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
        losses.append(float(loss))

        epochs.append(epoch)
        train_acc.append(get_accuracy(model, train_loader))
        valid_acc.append(get_accuracy(model, valid_loader))
        print("Epoch %d; Loss %f; Train Acc %f; Val Acc %f" % (
              epoch+1, loss, train_acc[-1], valid_acc[-1]))
    # plotting
    plt.title("Training Curve")
    plt.plot(losses, label="Train")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochs, train_acc, label="Train")
    plt.plot(epochs, valid_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

In [ ]:
model = TweetRNN(input_size=50, hidden_size=50, num_classes=len(interesting_labels.wanted_list))
train_loader = TweetBatcher(train, batch_size=64, drop_last=True)
valid_loader = TweetBatcher(valid, batch_size=64, drop_last=False)
train_rnn_network(model, train_loader, valid_loader, num_epochs=30, learning_rate=2e-4)
get_accuracy(model, test_loader)

### LSTM Model

In [ ]:
class TweetLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TweetLSTM, self).__init__()
        self.emb = nn.Embedding.from_pretrained(glove.vectors)
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Look up the embedding
        x = self.emb(x)
        # Set an initial hidden state and cell state
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        c0 = torch.zeros(1, x.size(0), self.hidden_size)
        # Forward propagate the LSTM
        out, _ = self.rnn(x, (h0, c0))
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
model_lstm = TweetLSTM(input_size=50, hidden_size=50, num_classes=len(interesting_labels.wanted_list))
train_rnn_network(model, train_loader, valid_loader, num_epochs=30, learning_rate=2e-4)
get_accuracy(model, test_loader)